In [10]:
#import
import numpy as np
import scipy.optimize as spo
import matplotlib.pyplot as plt
import math

In [29]:
def matvec(A,B,C,x):
    xN = x.shape[1]
    res = x.copy()
    for i in range(1,xN-1):
        res[:,i] = A[:,:,i].dot(x[:,i])+B[:,:,i-1].dot(x[:,i-1])+C[:,:,i].dot(x[:,i+1])
    res[:,0] = A[:,:,0].dot(x[:,0])+C[:,:,0].dot(x[:,1])
    res[:,-1] = B[:,:,-1].dot(x[:,-2])+A[:,:,-1].dot(x[:,-1])
    return res
    
def generate_problem(grid,x,A,B,C):

    f = x.copy()
    for i in range(x.shape[1]):
        for j in range(x.shape[0]):
            f[j,i] = np.exp(x[j,i])
    J = np.zeros((M,M,N))
    for i in range(N):
        for j in range(M):
            J[j][j][i] = np.exp(x[j][i])     
    return A,B,C,J,f

def solve_linear(A,B,C,d):
    x = np.empty(d.shape,dtype= np.float64)
    xN = x.shape[1]
    Y = np.empty(d.shape,dtype= np.float64)
    gamma = np.empty(C.shape,dtype= np.float64)
    ialpha= np.linalg.inv(A[:,:,0])
    gamma[:,:,0] = ialpha.dot(C[:,:,0])
    Y[:,0] = ialpha.dot(d[:,0])
    for i in range(1,xN-1):
        ialpha = np.linalg.inv(A[:,:,i]-B[:,:,i-1].dot(gamma[:,:,i-1]))
        gamma[:,:,i] = ialpha.dot(C[:,:,i])
        Y[:,i] = ialpha.dot(d[:,i]-B[:,:,i-1].dot(Y[:,i-1]))

    ialpha = np.linalg.inv(A[:,:,xN-1]-B[:,:,xN-2].dot(gamma[:,:,xN-2]))
    Y[:,xN-1] = ialpha.dot(d[:,xN-1]-B[:,:,xN-2].dot(Y[:,xN-2]))
    x[:,xN-1] = Y[:,xN-1]
    for i in reversed(range(xN-1)):
        x[:,i] = Y[:,i]-gamma[:,:,i].dot(x[:,i+1])
    return x

def calc_residual(A,B,C,d,x):
    return -(matvec(A,B,C,x)-d)

def newton_solver(A,B,C,J,d,x0,grid,r,eps = 1e-8, max_iter = 100):
    x = x0.copy()
    for i in range(max_iter):
        x += solve_linear(A+J,B,C,d+r)
        A,B,C,J,d = generate_problem(grid,x,A,B,C)
        resid = calc_residual(A,B,C,d+r,x)
        if(np.abs(resid).mean() < eps):
            break        
    return A,B,C,J,d,resid

In [31]:
N = 1000
M = 4 #in block

x= np.random.rand(M,N).astype(np.float64)
f = x.copy()
for i in range(x.shape[1]):
    for j in range(x.shape[0]):
        f[j,i] = np.exp(x[j,i])

J = np.zeros((M,M,N))

for i in range(N):
    for j in range(M):
        J[j][j][i] = np.exp(x[j][i])
    
B = np.random.rand(M,M,N)
C = np.random.rand(M,M,N)
C[:,:,0] = 0
A = np.random.rand(M,M,N)
A[:,:,-1] = 0

x0 = np.random.rand(M,N).astype(np.float64)
residual = f-matvec(A,B,C,x)

result = newton_solver(A, B, C, J, f, x0, None, residual)
print result

/home/lenar/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:15: RuntimeWarning: overflow encountered in exp
/home/lenar/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:19: RuntimeWarning: overflow encountered in exp


(array([[[ 0.3025014 ,  0.23417298,  0.90410845, ...,  0.56218244,
          0.64509859,  0.        ],
        [ 0.43020993,  0.94213482,  0.38379092, ...,  0.5554113 ,
          0.83240747,  0.        ],
        [ 0.91700861,  0.56984931,  0.77706648, ...,  0.45056456,
          0.4724777 ,  0.        ],
        [ 0.79394997,  0.16335483,  0.77293633, ...,  0.31896918,
          0.22324924,  0.        ]],

       [[ 0.06480644,  0.03388442,  0.37903071, ...,  0.25196796,
          0.12195237,  0.        ],
        [ 0.60174259,  0.07279684,  0.5459026 , ...,  0.79038154,
          0.03836821,  0.        ],
        [ 0.26329713,  0.79054903,  0.8496602 , ...,  0.5628256 ,
          0.43167631,  0.        ],
        [ 0.29426963,  0.7668464 ,  0.40682586, ...,  0.6205081 ,
          0.88842679,  0.        ]],

       [[ 0.03259414,  0.30301592,  0.15811707, ...,  0.71190799,
          0.25438103,  0.        ],
        [ 0.4727501 ,  0.39800051,  0.63777872, ...,  0.04563614,
          0